**_🚨 IMPORTANT: BEFORE RUNNING THE PROGRAM, SAVE THIS SHEET AS A CSV FILE! 🚨_**

---

| **Parameter**                               | **Value**                    |
|---------------------------------------------|------------------------------|
| **Start date**                              | 2010-01-01                   |
| **End date**                                | 2020-12-31                   |
| **Reference start date**                    | 2010-01-01                   |
| **Reference end date**                      | 2014-12-31                   |
| **Total fund capital (mm)**                 | 1000                         |
| **Standard deviation multiple**             | 3                            |

| **Global financing costs**                  |                              |
| Annual financing mean (%)                   | 0.50                         |
| Annual financing vol (%)                    | 0.10                         |
| Monthly spike probability                   | 0.02                         |
| Spike size (σ × multiplier)                 | 2.25                         |

| **Internal‐PA financing costs** _(optional)_ |                              |
| Internal financing mean (%)                 | *leave blank to use global*  |
| Internal financing vol (%)                  | *leave blank to use global*  |
| Internal spike probability                  | *leave blank to use global*  |
| Internal spike size (σ × multiplier)        | *leave blank to use global*  |

| **External‐PA financing costs** _(optional)_ |                              |
| External PA financing mean (%)              | *leave blank to use global*  |
| External PA financing vol (%)               | *leave blank to use global*  |
| External PA spike probability               | *leave blank to use global*  |
| External PA spike size (σ × multiplier)     | *leave blank to use global*  |

| **Active‐Extension financing costs** _(optional)_ |                          |
| ActiveExt financing mean (%)                | *leave blank to use global*  |
| ActiveExt financing vol (%)                 | *leave blank to use global*  |
| ActiveExt spike probability                 | *leave blank to use global*  |
| ActiveExt spike size (σ × multiplier)       | *leave blank to use global*  |

| **In‐House PA parameters**                  |                              |
| In-House annual return (%)                  | 4.00                         |
| In-House annual vol (%)                     | 1.00                         |
| In-House β                                   | 0.50                         |
| In-House α                                   | 0.50                         |

| **Extension (Active) α parameters**         |                              |
| Alpha-Extension annual return (%)           | 5.00                         |
| Alpha-Extension annual vol (%)              | 2.00                         |
| Active Extension capital (mm)               | 400                          |
| Active share (%) _(e.g. 50 for 150/50)_      | 50                           |

| **External PA α parameters**                |                              |
| External annual return (%)                  | 3.00                         |
| External annual vol (%)                     | 2.00                         |
| External PA capital (mm)                    | 300                          |
| External PA α fraction (%)                  | 50                           |

| **Correlations**                            |                              |
| Corr index–In-House                         | 0.05                         |
| Corr index–Alpha-Extension                  | 0.00                         |
| Corr index–External                         | 0.00                         |
| Corr In-House–Alpha-Extension               | 0.10                         |
| Corr In-House–External                      | 0.10                         |
| Corr Alpha-Extension–External               | 0.00                         |

| **Capital buckets**                         |                              |
| Internal PA capital (mm)                    | 300                          |
| External PA capital (mm)                    | 300                          |
| Active Extension capital (mm)               | 400                          |
| Total fund capital (mm) _(must equal sum)_  | 1000                         |

| **Buffer multiple (for breach test)**       | 3                            |

---

### How to Use:
1. **Fill in or adjust any “Value” cells**—especially the financing costs if they differ by bucket.
2. If you leave a bucket’s financing rows blank, the program will default to the **Global financing costs**.
3. **Save this sheet exactly as a CSV** (e.g. “parameters.csv”).
4. When you run `portable_alpha_model.py`, select this CSV when prompted.

Make sure every bold‐labeled “Parameter” appears—otherwise the loader will skip unknown rows. The order does not matter, as long as column headers remain `Parameter,Value`.


In [3]:
# portable_alpha_model.py

import csv
import tkinter as tk
from pathlib import Path
from tkinter import filedialog

import numpy as np
import pandas as pd

# =============================================================================
# 1. MAPPING: User-friendly labels → Internal variable names
# =============================================================================

LABEL_MAP = {
    "Number of simulations": "N_SIMULATIONS",
    "Simulation horizon (months)": "N_MONTHS",
    "Annual financing mean (%)": "financing_mean_annual",
    "Annual financing vol (%)": "financing_vol_annual",
    "Monthly spike probability": "spike_prob",
    "Spike size (σ × multiplier)": "spike_factor",
    "In-House annual return (%)": "mu_H",
    "In-House annual vol (%)": "sigma_H",
    "Alpha-Extension annual return (%)": "mu_E",
    "Alpha-Extension annual vol (%)": "sigma_E",
    "External annual return (%)": "mu_M",
    "External annual vol (%)": "sigma_M",
    "Corr index–In-House": "rho_idx_H",
    "Corr index–Alpha-Extension": "rho_idx_E",
    "Corr index–External": "rho_idx_M",
    "Corr In-House–Alpha-Extension": "rho_H_E",
    "Corr In-House–External": "rho_H_M",
    "Corr Alpha-Extension–External": "rho_E_M",
    "Buffer multiple": "buffer_multiple",
    # Continuous‐range parameters
    "X min (mm)": "X_min",
    "X max (mm)": "X_max",
    "X step (mm)": "X_step",
    "EM theta min": "EM_theta_min",
    "EM theta max": "EM_theta_max",
    "EM theta step": "EM_theta_step",
    # Backward compatibility if user still provides lists
    "X grid (mm)": "X_grid_list",
    "External manager α fractions": "EM_thetas_list",
    "In-House β": "w_beta_H",
    "In-House α": "w_alpha_H",
}

# =============================================================================
# 2. FILE-PICKER FOR CSV SELECTION
# =============================================================================

def select_csv_file():
    """
    Pop up a file-picker dialog so the user can choose a CSV file.
    Returns a pathlib.Path to the selected file.
    Raises FileNotFoundError if the user cancels.
    """
    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename(
        title="Select CSV File",
        filetypes=[("CSV files", "*.csv"), ("All files", "*.*")]
    )
    root.destroy()
    if not file_path:
        raise FileNotFoundError("No file selected.")
    return Path(file_path)

# =============================================================================
# 3. LOAD PARAMETERS USING MAPPING
# =============================================================================

def load_parameters(csv_filepath, label_map):
    """
    Read a CSV that may have leading instruction rows, then a header row "Parameter,Value".
    Skip all rows until the header, then parse friendly labels → internal names via label_map.
    Returns a dict {internal_var_name: parsed_value}.
    """
    params = {}

    # Read all lines
    lines = Path(csv_filepath).read_text(encoding="utf-8").splitlines()

    # Find header row that starts with "Parameter,"
    header_idx = None
    for i, line in enumerate(lines):
        if line.strip().startswith("Parameter,"):
            header_idx = i
            break

    if header_idx is None:
        raise ValueError(f"No header row starting with 'Parameter,' found in {csv_filepath}")

    # Only use header + following data lines
    header_and_data = lines[header_idx:]
    reader = csv.DictReader(header_and_data)

    for row in reader:
        friendly_key = row.get("Parameter", "").strip()
        if not friendly_key or friendly_key not in label_map:
            continue

        internal_key = label_map[friendly_key]
        raw_val = row.get("Value", "").strip()

        # If semicolon present, parse as list
        if ";" in raw_val:
            parts = [p.strip() for p in raw_val.split(";") if p.strip() != ""]
            parsed_list = []
            for p in parts:
                try:
                    if "." in p:
                        parsed_list.append(float(p))
                    else:
                        parsed_list.append(int(p))
                except ValueError:
                    parsed_list.append(p)
            params[internal_key] = parsed_list
        else:
            # Try int → float → string
            try:
                params[internal_key] = int(raw_val)
            except ValueError:
                try:
                    params[internal_key] = float(raw_val)
                except ValueError:
                    params[internal_key] = raw_val

    return params

# =============================================================================
# 4. HELPER TO LOAD INDEX RETURNS
# =============================================================================

def load_index_returns(csv_path):
    """
    Load a CSV of monthly index returns into a pandas Series.
    Expects a "Date" column and either "Monthly_TR" or "Return" column.
    Returns a pd.Series indexed by Date.
    """
    csv_path = Path(csv_path)
    if not csv_path.exists() or not csv_path.is_file():
        raise FileNotFoundError(f"Index CSV not found at {csv_path}")
    df = pd.read_csv(csv_path, parse_dates=["Date"])
    if "Date" not in df.columns:
        raise ValueError(f"'Date' column is missing from {csv_path}")
    if "Monthly_TR" in df.columns:
        col = "Monthly_TR"
    elif "Return" in df.columns:
        col = "Return"
    else:
        raise ValueError(f"CSV must contain a 'Monthly_TR' or 'Return' column: found {df.columns.tolist()}")

    df = df.sort_values("Date").reset_index(drop=True)
    df.set_index("Date", inplace=True)
    series = df[col].dropna().copy()
    series.index = pd.to_datetime(series.index)
    return series

# =============================================================================
# 5. SIMULATION AND UTILITY FUNCTIONS
# =============================================================================

def simulate_financing(T, financing_mean, financing_sigma, spike_prob, spike_factor):
    """
    Simulate a series of financing spreads f_t for T months,
    using a Normal + occasional jump model.
    """
    f = np.zeros(T)
    for t in range(T):
        base = financing_mean + np.random.normal(0, financing_sigma)
        jump = 0.0
        if np.random.rand() < spike_prob:
            jump = spike_factor * financing_sigma
        f[t] = max(base + jump, 0.0)
    return f

def build_cov_matrix(rho_idx_H, rho_idx_E, rho_idx_M,
                     rho_H_E, rho_H_M, rho_E_M,
                     idx_sigma, sigma_H, sigma_E, sigma_M):
    """
    Build the 4×4 covariance matrix for (Index, H, E, M).
    """
    sds = np.array([idx_sigma, sigma_H, sigma_E, sigma_M])
    rho = np.array([
        [1.0,       rho_idx_H, rho_idx_E, rho_idx_M],
        [rho_idx_H, 1.0,       rho_H_E,   rho_H_M],
        [rho_idx_E, rho_H_E,   1.0,       rho_E_M],
        [rho_idx_M, rho_H_M,   rho_E_M,   1.0    ]
    ])
    return np.outer(sds, sds) * rho

def simulate_alpha_streams(T, cov, mu_idx, mu_H, mu_E, mu_M):
    """
    Simulate T joint observations of (Index_return, H, E, M)
    from a multivariate Normal with given means and covariance.
    Returns an array of shape (T, 4).
    """
    means = np.array([mu_idx, mu_H, mu_E, mu_M])
    return np.random.multivariate_normal(means, cov, size=T)

def export_to_excel(inputs_dict, summary_df, raw_returns_dict, filename="Outputs.xlsx"):
    """
    Write inputs, summary, and raw returns into an Excel workbook.
    """
    with pd.ExcelWriter(filename, engine="openpyxl") as writer:
        # 1) Inputs sheet
        df_inputs = pd.DataFrame.from_dict(inputs_dict, orient="index", columns=["Value"])
        df_inputs.index.name = "Parameter"
        df_inputs.reset_index(inplace=True)
        df_inputs.to_excel(writer, sheet_name="Inputs", index=False)

        # 2) Summary sheet
        summary_df.to_excel(writer, sheet_name="Summary", index=False)

        # 3) Raw Returns sheets
        for sheet_name, df in raw_returns_dict.items():
            df.to_excel(writer, sheet_name=sheet_name, index=True)

    print(f"Exported results to {filename}")

# =============================================================================
# 6. MAIN EXECUTION
# =============================================================================

if __name__ == "__main__":
    # 6.1) Prompt user to select the parameters CSV
    try:
        params_csv_path = select_csv_file()
        print(f"Parameters CSV selected: {params_csv_path}")
    except FileNotFoundError:
        raise RuntimeError("No parameter CSV selected; exiting.")

    # 6.2) Load and unpack parameters
    raw_params = load_parameters(params_csv_path, LABEL_MAP)

    # Unpack scalar parameters (with defaults if missing)
    N_SIMULATIONS         = raw_params.get("N_SIMULATIONS", 5000)
    N_MONTHS              = raw_params.get("N_MONTHS", 12)
    financing_mean_annual = raw_params.get("financing_mean_annual", 0.005)
    financing_vol_annual  = raw_params.get("financing_vol_annual", 0.001)
    spike_prob            = raw_params.get("spike_prob", 0.02)
    spike_factor          = raw_params.get("spike_factor", 2.25)

    mu_H    = raw_params.get("mu_H", 0.04)
    sigma_H = raw_params.get("sigma_H", 0.01)
    mu_E    = raw_params.get("mu_E", 0.05)
    sigma_E = raw_params.get("sigma_E", 0.02)
    mu_M    = raw_params.get("mu_M", 0.03)
    sigma_M = raw_params.get("sigma_M", 0.02)

    rho_idx_H = raw_params.get("rho_idx_H", 0.05)
    rho_idx_E = raw_params.get("rho_idx_E", 0.00)
    rho_idx_M = raw_params.get("rho_idx_M", 0.00)
    rho_H_E   = raw_params.get("rho_H_E", 0.10)
    rho_H_M   = raw_params.get("rho_H_M", 0.10)
    rho_E_M   = raw_params.get("rho_E_M", 0.00)

    buffer_multiple = raw_params.get("buffer_multiple", 3.0)
    w_beta_H        = raw_params.get("w_beta_H", 0.50)
    w_alpha_H       = raw_params.get("w_alpha_H", 0.50)

    # Unpack continuous-range parameters (with defaults)
    X_min   = raw_params.get("X_min", 0)
    X_max   = raw_params.get("X_max", 1000)
    X_step  = raw_params.get("X_step", 100)

    EM_min  = raw_params.get("EM_theta_min", 0.50)
    EM_max  = raw_params.get("EM_theta_max", 0.75)
    EM_step = raw_params.get("EM_theta_step", 0.25)

    # Validate ranges
    if X_step <= 0:
        raise ValueError(f"X step must be positive; got {X_step}")
    if EM_step <= 0:
        raise ValueError(f"EM theta step must be positive; got {EM_step}")
    if X_max < X_min:
        raise ValueError(f"X max ({X_max}) < X min ({X_min})")
    if EM_max < EM_min:
        raise ValueError(f"EM theta max ({EM_max}) < EM theta min ({EM_min})")

    # Build continuous grids (inclusive of endpoint)
    X_grid = np.arange(X_min, X_max + X_step/2, X_step)
    EM_thetas = np.arange(EM_min, EM_max + EM_step/2, EM_step)

    # Warn if grid is too large
    total_combinations = len(X_grid) * len(EM_thetas)
    if total_combinations > 1000:
        print(
            f"WARNING: You will simulate {total_combinations} (X, θ) combinations. "
            "This may be computationally intensive. "
            "Consider increasing step sizes for a reasonable runtime."
        )

    # Backward-compatibility: if user provided explicit lists, use them instead
    if "X_grid_list" in raw_params:
        X_grid = raw_params["X_grid_list"]
    if "EM_thetas_list" in raw_params:
        EM_thetas = raw_params["EM_thetas_list"]

    # Convert annual percentages to monthly decimals
    financing_mean  = financing_mean_annual / 12
    financing_sigma = financing_vol_annual / 12
    mu_H_month      = mu_H / 12
    sigma_H_month   = sigma_H / 12
    mu_E_month      = mu_E / 12
    sigma_E_month   = sigma_E / 12
    mu_M_month      = mu_M / 12
    sigma_M_month   = sigma_M / 12

    # 6.3) Prompt user to select the index CSV
    print("Please select the INDEX CSV (monthly total returns).")
    try:
        INDEX_CSV_PATH = select_csv_file()
        print(f"Index CSV selected: {INDEX_CSV_PATH}")
    except FileNotFoundError:
        raise RuntimeError("Index CSV was not selected; exiting.")

    # 6.4) Load index returns
    try:
        idx_series = load_index_returns(INDEX_CSV_PATH)
        print(
            f"Loaded {len(idx_series)} months, "
            f"from {idx_series.index[0].date()} through {idx_series.index[-1].date()}."
        )
    except Exception as e:
        raise RuntimeError(f"Failed to load index returns: {e}")

    # Compute index mean and sigma (monthly)
    mu_idx    = idx_series.mean()
    idx_sigma = idx_series.std(ddof=1)

    # 6.5) Prepare inputs dictionary for Excel export
    inputs_dict = {
        "N_SIMULATIONS":          N_SIMULATIONS,
        "N_MONTHS":               N_MONTHS,
        "Financing Mean (annual)": financing_mean_annual,
        "Financing Vol  (annual)": financing_vol_annual,
        "Spike Probability (month)": spike_prob,
        "Spike Size (σ‐multiplier)": spike_factor,
        "μ_H (annual)":           mu_H,
        "σ_H (annual)":           sigma_H,
        "μ_E (annual)":           mu_E,
        "σ_E (annual)":           sigma_E,
        "μ_M (annual)":           mu_M,
        "σ_M (annual)":           sigma_M,
        "ρ_idx_H":                rho_idx_H,
        "ρ_idx_E":                rho_idx_E,
        "ρ_idx_M":                rho_idx_M,
        "ρ_H_E":                  rho_H_E,
        "ρ_H_M":                  rho_H_M,
        "ρ_E_M":                  rho_E_M,
        "Buffer multiple (m)":    buffer_multiple,
        "In‐House w_beta":        w_beta_H,
        "In‐House w_alpha":       w_alpha_H,
        "X_min (mm)":             X_min,
        "X_max (mm)":             X_max,
        "X_step (mm)":            X_step,
        "EM theta_min":           EM_min,
        "EM theta_max":           EM_max,
        "EM theta_step":          EM_step
    }

    # 6.6) Run Monte Carlo simulation for all scenarios
    all_summaries = []
    all_raw_returns = {}

    for X in X_grid:
        # 6.6.1) Simulate financing spreads
        f_series = simulate_financing(N_MONTHS, financing_mean, financing_sigma, spike_prob, spike_factor)
        dates_sim = pd.date_range(
            start=idx_series.index[-1] + pd.DateOffset(months=1),
            periods=N_MONTHS, freq="ME"
        )

        # 6.6.2) Pre-allocate one-year returns
        results = {"Base": np.zeros(N_SIMULATIONS)}
        for theta in EM_thetas:
            results[f"EM_X={X}_θ={theta}"] = np.zeros(N_SIMULATIONS)
        results[f"EXT_X={X}"] = np.zeros(N_SIMULATIONS)

        # 6.6.3) Build covariance matrix for this run
        cov_mat = build_cov_matrix(
            rho_idx_H, rho_idx_E, rho_idx_M,
            rho_H_E, rho_H_M, rho_E_M,
            idx_sigma, sigma_H_month, sigma_E_month, sigma_M_month
        )

        # 6.6.4) Draw all streams for all sims at once
        sims = np.random.multivariate_normal(
            [mu_idx, mu_H_month, mu_E_month, mu_M_month],
            cov_mat,
            size=(N_SIMULATIONS, N_MONTHS)
        )  # shape: (N_SIMULATIONS, N_MONTHS, 4)

        # Precompute one-year index returns for each simulation
        idx_one_year = np.prod(1 + sims[:, :, 0], axis=1) - 1

        # 6.6.5) Tile financing series
        f_matrix = np.tile(f_series, (N_SIMULATIONS, 1))

        # 6.6.6) Prepare raw_returns DataFrames for first simulation paths
        raw_returns = {
            "Base": pd.DataFrame(index=dates_sim),
            f"EXT_X={X}": pd.DataFrame(index=dates_sim),
        }
        for theta in EM_thetas:
            raw_returns[f"EM_X={X}_θ={theta}"] = pd.DataFrame(index=dates_sim)

        # 6.6.7) Loop over simulations
        for sim in range(N_SIMULATIONS):
            r_beta = sims[sim, :, 0]
            r_H    = sims[sim, :, 1]
            r_E    = sims[sim, :, 2]
            r_M    = sims[sim, :, 3]
            f_t    = f_matrix[sim, :]

            # Base strategy
            R_base = (r_beta - f_t) * w_beta_H + r_H * w_alpha_H
            results["Base"][sim] = np.prod(1 + R_base) - 1

            # EXT strategy
            inhouse_beta_factor  = (1.0 - X/1000.0) / 2.0
            inhouse_alpha_factor = (1.0 - X/1000.0) / 2.0
            ext_beta_factor      = X / 1000.0
            ext_alpha_factor     = X / 1000.0
            R_ext = (
                (r_beta - f_t) * (inhouse_beta_factor + ext_beta_factor)
                + r_H * inhouse_alpha_factor
                + r_E * ext_alpha_factor
            )
            results[f"EXT_X={X}"][sim] = np.prod(1 + R_ext) - 1

            # EM scenarios
            for theta in EM_thetas:
                em_beta_factor  = X / 1000.0
                em_alpha_factor = theta * (X / 1000.0)
                R_em = (
                    (r_beta - f_t) * (inhouse_beta_factor + em_beta_factor)
                    + r_H * inhouse_alpha_factor
                    + r_M * em_alpha_factor
                )
                results[f"EM_X={X}_θ={theta}"][sim] = np.prod(1 + R_em) - 1

            # Save monthly path for first simulation
            if sim == 0:
                raw_returns["Base"] = pd.DataFrame({"Base": R_base}, index=dates_sim)
                raw_returns[f"EXT_X={X}"] = pd.DataFrame({f"EXT_X={X}": R_ext}, index=dates_sim)
                for theta in EM_thetas:
                    raw_returns[f"EM_X={X}_θ={theta}"] = pd.DataFrame(
                        {f"EM_X={X}_θ={theta}": R_em}, index=dates_sim
                    )

        # 6.6.8) Build yearly results DataFrame
        df_yearly = pd.DataFrame(results)

        # 6.6.9) Compute summary metrics
        summary_rows = []
        for config, arr in df_yearly.items():
            ann_ret = np.mean(arr)
            ann_vol = np.std(arr, ddof=1)
            var_95  = np.percentile(arr, 5)
            # Tracking error = std of (strategy return – index return)
            active_returns = arr - idx_one_year
            te = np.std(active_returns, ddof=1)

            if config not in raw_returns:
                raise KeyError(f"Config '{config}' not found. Keys: {list(raw_returns.keys())}")
            mr_series = raw_returns[config].iloc[:, 0]
            threshold = -buffer_multiple * idx_sigma
            breach_pct = np.mean(mr_series < threshold) * 100

            summary_rows.append({
                "Config": config,
                "X (mm)": X,
                "Ann Return": ann_ret,
                "Ann Vol": ann_vol,
                "VaR 95": var_95,
                "TE (est.)": te,
                "Breach %": breach_pct
            })

        summary_df = pd.DataFrame(summary_rows)
        all_summaries.append(summary_df)

        # 6.6.10) Collect raw returns for Excel sheets
        for key, df in raw_returns.items():
            all_raw_returns[key] = df

    # 6.7) Combine all summaries and export to Excel
    final_summary = pd.concat(all_summaries, ignore_index=True)
    export_to_excel(inputs_dict, final_summary, all_raw_returns)

    # ─── HUMAN‐FRIENDLY POST‐PROCESSING ───────────────────────────────────────────

    # 1) Make a copy so we don’t overwrite the original
    display_df = final_summary.copy()

    # 2) Rename columns to more explicit, human‐readable labels
    display_df = display_df.rename(columns={
        "X (mm)":        "Cash Buffer (USD mm)",
        "Ann Return":    "Annual Return (%)",
        "Ann Vol":       "Annual Volatility (%)",
        "VaR 95":        "95%-VaR (%)",
        "TE (est.)":     "Tracking Error (%)",
        "Breach %":      "Breach Probability (%)"
    })

    # 3) Convert numeric columns into percentage strings (except Cash Buffer)
    pct_cols = [
        "Annual Return (%)",
        "Annual Volatility (%)",
        "95%-VaR (%)",
        "Tracking Error (%)",
        "Breach Probability (%)"
    ]

    for col in pct_cols:
        display_df[col] = (display_df[col]).map("{:.1f}%".format)

    # 4) Show the nicely formatted table
    print("\n=== Summary Table (Human‐Friendly) ===\n")
    print(display_df.to_string(index=False))

    # 5) Print a short English description for each row
    print("\n=== Narrative Summaries ===\n")
    for _, row in display_df.iterrows():
        cfg   = row["Config"]
        x_mm  = row["Cash Buffer (USD mm)"]
        ret   = row["Annual Return (%)"]
        vol   = row["Annual Volatility (%)"]
        var95 = row["95%-VaR (%)"]
        te    = row["Tracking Error (%)"]
        br    = row["Breach Probability (%)"]

        print(
            f"For configuration '{cfg}' with a cash buffer of ${x_mm:.0f} mm:\n"
            f"  • Expected annual return: {ret}\n"
            f"  • Annual volatility: {vol}\n"
            f"  • 95% Value‐at‐Risk: {var95}\n"
            f"  • Estimated tracking error: {te}\n"
            f"  • Probability of breaching the buffer: {br}\n"
        )


Parameters CSV selected: /Users/teacher/Library/CloudStorage/Dropbox/Learning/Code/Portable Alpha-Extension Model/parameters.csv
Please select the INDEX CSV (monthly total returns).
Index CSV selected: /Users/teacher/Library/CloudStorage/Dropbox/Learning/Code/Portable Alpha-Extension Model/sp500tr_fred_divyield.csv
Loaded 663 months, from 1970-01-01 through 2025-03-01.
Exported results to Outputs.xlsx

=== Summary Table (Human‐Friendly) ===

            Config  Cash Buffer (USD mm) Annual Return (%) Annual Volatility (%) 95%-VaR (%) Tracking Error (%) Breach Probability (%)
              Base                   0.0              4.5%                  0.8%        3.3%               0.7%                   0.0%
    EM_X=0.0_θ=0.5                   0.0              4.5%                  0.8%        3.3%               0.7%                   0.0%
   EM_X=0.0_θ=0.75                   0.0              4.5%                  0.8%        3.3%               0.7%                   0.0%
         EXT_X

In [4]:
# portable_alpha_model.py


import numpy as np
import pandas as pd

# =============================================================================
# 1. MAPPING: User-friendly labels → Internal variable names
# =============================================================================
LABEL_MAP = {
    # Existing parameter mappings
    "Number of simulations":           "N_SIMULATIONS",
    "Simulation horizon (months)":     "N_MONTHS",
    "Annual financing mean (%)":       "financing_mean_annual",
    "Annual financing vol (%)":        "financing_vol_annual",
    "Monthly spike probability":       "spike_prob",
    "Spike size (σ × multiplier)":     "spike_factor",
    "In-House annual return (%)":      "mu_H",
    "In-House annual vol (%)":         "sigma_H",
    "Alpha-Extension annual return (%)":"mu_E",
    "Alpha-Extension annual vol (%)":  "sigma_E",
    "External annual return (%)":      "mu_M",
    "External annual vol (%)":         "sigma_M",
    "Corr index–In-House":             "rho_idx_H",
    "Corr index–Alpha-Extension":      "rho_idx_E",
    "Corr index–External":             "rho_idx_M",
    "Corr In-House–Alpha-Extension":   "rho_H_E",
    "Corr In-House–External":          "rho_H_M",
    "Corr Alpha-Extension–External":   "rho_E_M",
    "Buffer multiple":                 "buffer_multiple",

    # Reference period keys
    "Start date":             "start_date",        # analysis window begin
    "End date":               "end_date",          # analysis window end
    "Reference start date":   "ref_start_date",    # window for σ_ref
    "Reference end date":     "ref_end_date",

    # Capital bucket keys
    "Total fund capital (mm)":         "total_fund_capital",
    "Standard deviation multiple":     "sd_of_vol_mult",
    "External PA capital (mm)":        "external_pa_capital",
    "External PA α fraction (%)":      "external_pa_alpha_frac",
    "Active Extension capital (mm)":   "active_ext_capital",
    "Active share (%)":                "active_share",
    "Internal PA capital (mm)":        "internal_pa_capital",

    # Backward-compatibility (if user still uses X_grid/EM_thetas)
    "X grid (mm)":                     "X_grid_list",
    "External manager α fractions":    "EM_thetas_list",
    "In-House β":                      "w_beta_H",
    "In-House α":                      "w_alpha_H",
}

# =============================================================================
# 2. FILE-PICKER FOR CSV SELECTION
# =============================================================================
def select_csv_file():
    """
    Pop up a file-picker dialog so the user can choose a CSV file.
    Returns a pathlib.Path to the selected file.
    Raises FileNotFoundError if the user cancels.
    """
    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename(
        title="Select CSV File",
        filetypes=[("CSV files", "*.csv"), ("All files", "*.*")]
    )
    root.destroy()
    if not file_path:
        raise FileNotFoundError("No file selected.")
    return Path(file_path)

# =============================================================================
# 3. LOAD PARAMETERS USING MAPPING
# =============================================================================
def load_parameters(csv_filepath, label_map):
    """
    Read a CSV that may have leading instruction rows, then a header row "Parameter,Value".
    Skip all rows until the header, then parse friendly labels → internal names via label_map.
    Returns a dict {internal_var_name: parsed_value}.
    """
    params = {}
    lines = Path(csv_filepath).read_text(encoding="utf-8").splitlines()

    # Find the header row that begins with "Parameter,"
    header_idx = None
    for i, line in enumerate(lines):
        if line.strip().startswith("Parameter,"):
            header_idx = i
            break
    if header_idx is None:
        raise ValueError(f"No header row starting with 'Parameter,' found in {csv_filepath}")

    header_and_data = lines[header_idx:]
    reader = csv.DictReader(header_and_data)

    for row in reader:
        friendly_key = row.get("Parameter", "").strip()
        if not friendly_key or friendly_key not in label_map:
            continue
        internal_key = label_map[friendly_key]
        raw_val = row.get("Value", "").strip()

        # If semicolon present, parse as list
        if ";" in raw_val:
            parts = [p.strip() for p in raw_val.split(";") if p.strip() != ""]
            parsed_list = []
            for p in parts:
                try:
                    if "." in p:
                        parsed_list.append(float(p))
                    else:
                        parsed_list.append(int(p))
                except ValueError:
                    parsed_list.append(p)
            params[internal_key] = parsed_list
        else:
            # Try int → float → string
            try:
                params[internal_key] = int(raw_val)
            except ValueError:
                try:
                    params[internal_key] = float(raw_val)
                except ValueError:
                    params[internal_key] = raw_val

    return params

# =============================================================================
# 4. HELPER TO LOAD INDEX RETURNS
# =============================================================================
def load_index_returns(csv_path):
    """
    Load a CSV of monthly index returns into a pandas Series.
    Expects a "Date" column and either "Monthly_TR" or "Return" column.
    Returns a pd.Series indexed by Date.
    """
    csv_path = Path(csv_path)
    if not csv_path.exists() or not csv_path.is_file():
        raise FileNotFoundError(f"Index CSV not found at {csv_path}")
    df = pd.read_csv(csv_path, parse_dates=["Date"])
    if "Date" not in df.columns:
        raise ValueError(f"'Date' column is missing from {csv_path}")
    if "Monthly_TR" in df.columns:
        col = "Monthly_TR"
    elif "Return" in df.columns:
        col = "Return"
    else:
        raise ValueError(f"CSV must contain 'Monthly_TR' or 'Return': found {df.columns.tolist()}")

    df = df.sort_values("Date").reset_index(drop=True)
    df.set_index("Date", inplace=True)
    series = df[col].dropna().copy()
    series.index = pd.to_datetime(series.index)
    return series

# =============================================================================
# 5. SIMULATION AND UTILITY FUNCTIONS
# =============================================================================
def simulate_financing(T, financing_mean, financing_sigma, spike_prob, spike_factor):
    """
    Simulate a series of financing spreads f_t for T months,
    using a Normal + occasional jump model.
    """
    f = np.zeros(T)
    for t in range(T):
        base = financing_mean + np.random.normal(0, financing_sigma)
        jump = 0.0
        if np.random.rand() < spike_prob:
            jump = spike_factor * financing_sigma
        f[t] = max(base + jump, 0.0)
    return f

def build_cov_matrix(rho_idx_H, rho_idx_E, rho_idx_M,
                     rho_H_E, rho_H_M, rho_E_M,
                     idx_sigma, sigma_H, sigma_E, sigma_M):
    """
    Build the 4×4 covariance matrix for (Index, H, E, M).
      –  index (r_beta)
      –  in-house alpha (r_H)
      –  extension alpha (r_E)
      –  external PA alpha (r_M)
    """
    sds = np.array([idx_sigma, sigma_H, sigma_E, sigma_M])
    rho = np.array([
        [1.0,       rho_idx_H, rho_idx_E, rho_idx_M],
        [rho_idx_H, 1.0,       rho_H_E,   rho_H_M],
        [rho_idx_E, rho_H_E,   1.0,       rho_E_M],
        [rho_idx_M, rho_H_M,   rho_E_M,   1.0    ]
    ])
    return np.outer(sds, sds) * rho

def simulate_alpha_streams(T, cov, mu_idx, mu_H, mu_E, mu_M):
    """
    Simulate T joint observations of (r_beta, r_H, r_E, r_M)
    from a multivariate normal with given means and covariance.
    Returns an array of shape (T, 4).
    """
    means = np.array([mu_idx, mu_H, mu_E, mu_M])
    return np.random.multivariate_normal(means, cov, size=T)

def export_to_excel(inputs_dict, summary_df, raw_returns_dict, filename="Outputs.xlsx"):
    """
    Write inputs, summary, and raw returns into an Excel workbook.
    """
    with pd.ExcelWriter(filename, engine="openpyxl") as writer:
        # 1) Inputs sheet
        df_inputs = pd.DataFrame.from_dict(inputs_dict, orient="index", columns=["Value"])
        df_inputs.index.name = "Parameter"
        df_inputs.reset_index(inplace=True)
        df_inputs.to_excel(writer, sheet_name="Inputs", index=False)

        # 2) Summary sheet
        summary_df.to_excel(writer, sheet_name="Summary", index=False)

        # 3) Raw Returns sheets
        for sheet_name, df in raw_returns_dict.items():
            df.to_excel(writer, sheet_name=sheet_name, index=True)

    print(f"Exported results to {filename}")

# =============================================================================
# 6. MAIN EXECUTION
# =============================================================================
if __name__ == "__main__":
    # 6.1) Prompt user to select the parameters CSV
    try:
        params_csv_path = select_csv_file()
        print(f"Parameters CSV selected: {params_csv_path}")
    except FileNotFoundError:
        raise RuntimeError("No parameter CSV selected; exiting.")

    # 6.2) Load and unpack parameters
    raw_params = load_parameters(params_csv_path, LABEL_MAP)

    # ── A) Unpack “reference” period for index-vol calculation ────────────────
    ref_start_str  = raw_params.get("ref_start_date", None)
    ref_end_str    = raw_params.get("ref_end_date",   None)
    ref_start_date = pd.to_datetime(ref_start_str) if ref_start_str else None
    ref_end_date   = pd.to_datetime(ref_end_str)   if ref_end_str   else None

    # ── B) Unpack “analysis window” for index history ───────────────────────
    start_str = raw_params.get("start_date", None)
    end_str   = raw_params.get("end_date",   None)
    start_date = pd.to_datetime(start_str) if start_str else None
    end_date   = pd.to_datetime(end_str)   if end_str   else None

    # ── C) Unpack all the other parameters as before ─────────────────────────
    N_SIMULATIONS         = raw_params.get("N_SIMULATIONS", 5000)
    N_MONTHS              = raw_params.get("N_MONTHS", 12)
    financing_mean_annual = raw_params.get("financing_mean_annual", 0.005)
    financing_vol_annual  = raw_params.get("financing_vol_annual", 0.001)
    spike_prob            = raw_params.get("spike_prob", 0.02)
    spike_factor          = raw_params.get("spike_factor", 2.25)

    mu_H    = raw_params.get("mu_H", 0.04)
    sigma_H = raw_params.get("sigma_H", 0.01)
    mu_E    = raw_params.get("mu_E", 0.05)
    sigma_E = raw_params.get("sigma_E", 0.02)
    mu_M    = raw_params.get("mu_M", 0.03)
    sigma_M = raw_params.get("sigma_M", 0.02)

    rho_idx_H = raw_params.get("rho_idx_H", 0.05)
    rho_idx_E = raw_params.get("rho_idx_E", 0.00)
    rho_idx_M = raw_params.get("rho_idx_M", 0.00)
    rho_H_E   = raw_params.get("rho_H_E", 0.10)
    rho_H_M   = raw_params.get("rho_H_M", 0.10)
    rho_E_M   = raw_params.get("rho_E_M", 0.00)

    buffer_multiple = raw_params.get("buffer_multiple", 3.0)
    w_beta_H        = raw_params.get("w_beta_H", 0.50)
    w_alpha_H       = raw_params.get("w_alpha_H", 0.50)

    # ── D) Unpack the new “capital bucket” parameters ─────────────────────────
    total_fund_capital       = raw_params.get("total_fund_capital", 1000)    # in $ mm
    sd_of_vol_mult           = raw_params.get("sd_of_vol_mult",    3)       # e.g. “3× vol” for margin
    external_pa_capital      = raw_params.get("external_pa_capital",   0)   # $ mm
    external_pa_alpha_frac   = raw_params.get("external_pa_alpha_frac", 50) # percent by default
    external_pa_alpha_frac  /= 100.0  # convert percent → decimal (0.50)

    active_ext_capital       = raw_params.get("active_ext_capital",  0)     # $ mm
    active_share_percent     = raw_params.get("active_share",       50)    # percent by default (50% ⇒ 150/50 program)
    active_share             = active_share_percent / 100.0  # convert % → decimal

    internal_pa_capital      = raw_params.get("internal_pa_capital", 0)     # $ mm

    # ── E) Check that the three buckets sum to total_fund_capital ────────────
    sum_buckets = external_pa_capital + active_ext_capital + internal_pa_capital
    if abs(sum_buckets - total_fund_capital) > 1e-6:
        raise RuntimeError(
            f"Buckets must sum to {total_fund_capital} mm. "
            f"Given: ExtPA={external_pa_capital}, ActExt={active_ext_capital}, InPA={internal_pa_capital} → sum={sum_buckets} mm."
        )

    # Convert annual percentages to monthly decimals
    financing_mean  = financing_mean_annual / 12
    financing_sigma = financing_vol_annual / 12
    mu_H_month      = mu_H / 12
    sigma_H_month   = sigma_H / 12
    mu_E_month      = mu_E / 12
    sigma_E_month   = sigma_E / 12
    mu_M_month      = mu_M / 12
    sigma_M_month   = sigma_M / 12

    # 6.3) Prompt user to select the INDEX CSV
    print("Please select the INDEX CSV (monthly total returns).")
    try:
        INDEX_CSV_PATH = select_csv_file()
        print(f"Index CSV selected: {INDEX_CSV_PATH}")
    except FileNotFoundError:
        raise RuntimeError("Index CSV was not selected; exiting.")

    # 6.4) Load full index-returns series
    try:
        idx_full = load_index_returns(INDEX_CSV_PATH)
        print(f"Loaded {len(idx_full)} months from the raw index CSV.")
    except Exception as e:
        raise RuntimeError(f"Failed to load index returns: {e}")

    # ── F) Filter idx_full → idx_series by [start_date : end_date] for simulation ─
    if start_date or end_date:
        sd = start_date if start_date else idx_full.index.min()
        ed = end_date   if end_date   else idx_full.index.max()
        idx_series = idx_full.loc[(idx_full.index >= sd) & (idx_full.index <= ed)]
        if len(idx_series) == 0:
            raise RuntimeError(f"No index data between {sd.date()} and {ed.date()}.")
        print(f"Using idx_series from {idx_series.index[0].date()} to {idx_series.index[-1].date()} (n={len(idx_series)})")
    else:
        idx_series = idx_full.copy()
        print(f"Using full idx_series: {idx_series.index[0].date()}–{idx_series.index[-1].date()}")

    # ── G) Filter idx_full → idx_ref by [ref_start_date : ref_end_date] for σ_ref calculation ─
    if ref_start_date or ref_end_date:
        rstart = ref_start_date if ref_start_date else idx_full.index.min()
        rend   = ref_end_date   if ref_end_date   else idx_full.index.max()
        idx_ref = idx_full.loc[(idx_full.index >= rstart) & (idx_full.index <= rend)]
        if len(idx_ref) == 0:
            raise RuntimeError(f"No index data in reference window [{rstart.date()}–{rend.date()}].")
        print(f"Using idx_ref from {idx_ref.index[0].date()} to {idx_ref.index[-1].date()} (n={len(idx_ref)})")
    else:
        idx_ref = idx_full.copy()
        print("No reference window given; using full idx_ref range.")

    # Compute means and volatilities
    mu_idx      = idx_series.mean()            # monthly
    idx_sigma   = idx_series.std(ddof=1)       # monthly
    idx_ref_vol = idx_ref.std(ddof=1)          # monthly

    print(f"Analysis-window: μ_idx = {mu_idx:.4f}, σ_idx = {idx_sigma:.4f}")
    print(f"Reference-window: σ_ref = {idx_ref_vol:.4f}")

    # ── H) Compute how much cash/margin is needed to back $1 b of beta ─────────
    internal_beta_backing = idx_ref_vol * sd_of_vol_mult * total_fund_capital  # in $ mm
    print(f"Margin-backing needed for $1 b beta: ${internal_beta_backing:.1f} mm")

    # The remainder of “internal” beyond margin-backing is in-house PA:
    internal_cash_leftover = total_fund_capital - internal_beta_backing - internal_pa_capital
    if internal_cash_leftover < -1e-6:
        raise RuntimeError(
            f"Not enough capital: margin_bac ({internal_beta_backing:.1f}) + internal_PA ({internal_pa_capital:.1f}) "
            f"exceeds {total_fund_capital:.1f}.")
    print(f"After backing beta (${internal_beta_backing:.1f} mm) and in-house PA "
          f"(${internal_pa_capital:.1f} mm), internal cash leftover = ${internal_cash_leftover:.1f} mm.")

    # 6.5) Prepare inputs dictionary for Excel export (include all new variables)
    inputs_dict = {
        "N_SIMULATIONS":                 N_SIMULATIONS,
        "N_MONTHS":                      N_MONTHS,
        "Financing Mean (annual)":       financing_mean_annual,
        "Financing Vol  (annual)":       financing_vol_annual,
        "Spike Probability (month)":     spike_prob,
        "Spike Size (σ‐multiplier)":     spike_factor,
        "μ_H (annual)":                  mu_H,
        "σ_H (annual)":                  sigma_H,
        "μ_E (annual)":                  mu_E,
        "σ_E (annual)":                  sigma_E,
        "μ_M (annual)":                  mu_M,
        "σ_M (annual)":                  sigma_M,
        "ρ_idx_H":                       rho_idx_H,
        "ρ_idx_E":                       rho_idx_E,
        "ρ_idx_M":                       rho_idx_M,
        "ρ_H_E":                         rho_H_E,
        "ρ_H_M":                         rho_H_M,
        "ρ_E_M":                         rho_E_M,
        "Buffer multiple (m)":           buffer_multiple,
        "In‐House w_beta":               w_beta_H,
        "In‐House w_alpha":              w_alpha_H,
        "Start date":                    start_date.date()    if start_date else "",
        "End date":                      end_date.date()      if end_date else "",
        "Reference start date":          ref_start_date.date() if ref_start_date else "",
        "Reference end date":            ref_end_date.date()   if ref_end_date else "",
        "Total fund capital (mm)":       total_fund_capital,
        "SD of Vol multiple":            sd_of_vol_mult,
        "External PA capital (mm)":      external_pa_capital,
        "External PA α fraction (%)":    external_pa_alpha_frac * 100,
        "Active Extension capital (mm)": active_ext_capital,
        "Active share (%)":              active_share * 100,
        "Internal PA capital (mm)":      internal_pa_capital,
        "Margin‐backing (mm)":           internal_beta_backing,
        "Internal cash leftover (mm)":    internal_cash_leftover
    }

    # 6.6) RUN MONTE CARLO SIMULATION FOR the SINGLE SCENARIO just defined
    all_summaries = []
    all_raw_returns = {}

    # ── 6.6.1) Simulate financing spreads
    f_series = simulate_financing(N_MONTHS,
                                  financing_mean,
                                  financing_sigma,
                                  spike_prob,
                                  spike_factor)
    dates_sim = pd.date_range(
        start=idx_series.index[-1] + pd.DateOffset(months=1),
        periods=N_MONTHS, freq="ME"
    )

    # ── 6.6.2) Pre-allocate one-year returns
    results = {
        "Base":       np.zeros(N_SIMULATIONS),
        "ExternalPA": np.zeros(N_SIMULATIONS),
        "ActiveExt":  np.zeros(N_SIMULATIONS),
    }

    # ── 6.6.3) Build covariance matrix using reference vol for index
    cov_mat = build_cov_matrix(
        rho_idx_H, rho_idx_E, rho_idx_M,
        rho_H_E, rho_H_M, rho_E_M,
        idx_ref_vol,   # index vol = σ_ref
        sigma_H_month,
        sigma_E_month,
        sigma_M_month
    )

    # ── 6.6.4) Draw all streams for all sims at once:
    # sims[...,0] = r_beta; sims[...,1] = r_H; sims[...,2] = r_E; sims[...,3] = r_M
    sims = np.random.multivariate_normal(
        [mu_idx, mu_H_month, mu_E_month, mu_M_month],
        cov_mat,
        size=(N_SIMULATIONS, N_MONTHS)
    )  # shape: (N_SIMULATIONS, N_MONTHS, 4)

    # Precompute one-year index returns for each simulation
    idx_one_year = np.prod(1 + sims[:, :, 0], axis=1) - 1

    # ── 6.6.5) Tile financing series
    f_matrix = np.tile(f_series, (N_SIMULATIONS, 1))

    # ── 6.6.6) Prepare raw_returns DataFrames for the first sim path
    raw_returns = {
        "Base":       pd.DataFrame(index=dates_sim),
        "ExternalPA": pd.DataFrame(index=dates_sim),
        "ActiveExt":  pd.DataFrame(index=dates_sim),
    }

    # ── 6.6.7) Loop over simulations
    for sim in range(N_SIMULATIONS):
        r_beta = sims[sim, :, 0]
        r_H    = sims[sim, :, 1]
        r_E    = sims[sim, :, 2]
        r_M    = sims[sim, :, 3]
        f_t    = f_matrix[sim, :]

        # ——————————— BASE (fully internal) ———————————
        R_base = (r_beta - f_t) * w_beta_H + r_H * w_alpha_H
        results["Base"][sim] = np.prod(1 + R_base) - 1

        # ———————— EXTERNAL PA return formula —————————
        # R_ExtPA_t = (X/1000)*(r_beta - f_t) + (X/1000)*external_pa_alpha_frac*(r_M)
        w_ext_pa = external_pa_capital / total_fund_capital
        R_extpa_t = w_ext_pa * (r_beta - f_t) + w_ext_pa * external_pa_alpha_frac * r_M
        results["ExternalPA"][sim] = np.prod(1 + R_extpa_t) - 1

        # ———————— ACTIVE EXTENSION return formula —————————
        # R_ActExt_t = (Y/1000)*(r_beta - f_t) + (Y/1000)*active_share*(r_E)
        w_act_ext = active_ext_capital / total_fund_capital
        R_actext_t = w_act_ext * (r_beta - f_t) + w_act_ext * active_share * r_E
        results["ActiveExt"][sim] = np.prod(1 + R_actext_t) - 1

        # — Save monthly path for sim == 0 so we can compute “breach” later —
        if sim == 0:
            raw_returns["Base"] = pd.DataFrame({"Base": R_base}, index=dates_sim)
            raw_returns["ExternalPA"] = pd.DataFrame({"ExternalPA": R_extpa_t}, index=dates_sim)
            raw_returns["ActiveExt"]  = pd.DataFrame({"ActiveExt": R_actext_t}, index=dates_sim)

    # ── 6.6.8) Build yearly results DataFrame
    df_yearly = pd.DataFrame(results)

    # ── 6.6.9) Compute summary metrics
    summary_rows = []
    for config, arr in df_yearly.items():
        ann_ret = np.mean(arr)
        ann_vol = np.std(arr, ddof=1)
        var_95  = np.percentile(arr, 5)
        # Tracking error = std of (bucket_return - index_return)
        active_returns = arr - idx_one_year
        te = np.std(active_returns, ddof=1)

        if config not in raw_returns:
            raise KeyError(f"Config '{config}' not found in raw_returns.")
        mr_series = raw_returns[config].iloc[:, 0]
        threshold = -buffer_multiple * idx_sigma
        breach_pct = np.mean(mr_series < threshold) * 100

        summary_rows.append({
            "Config":     config,
            "Ann Return": ann_ret,
            "Ann Vol":    ann_vol,
            "VaR 95":     var_95,
            "TE (est.)":  te,
            "Breach %":   breach_pct
        })

    summary_df = pd.DataFrame(summary_rows)
    all_summaries.append(summary_df)

    # ── 6.6.10) Collect raw returns for Excel sheets
    for key, df in raw_returns.items():
        all_raw_returns[key] = df

    # 6.7) Combine summaries and export
    final_summary = pd.concat(all_summaries, ignore_index=True)
    export_to_excel(inputs_dict, final_summary, all_raw_returns)

    # ── HUMAN-FRIENDLY POST-PROCESSING ──────────────────────────────────────
    display_df = final_summary.copy()
    display_df = display_df.rename(columns={
        "Ann Return": "Annual Return (%)",
        "Ann Vol":    "Annual Volatility (%)",
        "VaR 95":     "95%-VaR (%)",
        "TE (est.)":  "Tracking Error (%)",
        "Breach %":   "Breach Probability (%)"
    })
    pct_cols = [
        "Annual Return (%)",
        "Annual Volatility (%)",
        "95%-VaR (%)",
        "Tracking Error (%)",
        "Breach Probability (%)"
    ]
    for col in pct_cols:
        display_df[col] = display_df[col].map("{:.1f}%".format)

    print("\n=== Summary Table (Human-Friendly) ===\n")
    print(display_df.to_string(index=False))

    print("\n=== Narrative Summaries ===\n")
    for _, row in display_df.iterrows():
        cfg   = row["Config"]
        ret   = row["Annual Return (%)"]
        vol   = row["Annual Volatility (%)"]
        var95 = row["95%-VaR (%)"]
        te    = row["Tracking Error (%)"]
        br    = row["Breach Probability (%)"]
        print(
            f"For '{cfg}':\n"
            f"  • Expected annual return: {ret}\n"
            f"  • Annual volatility: {vol}\n"
            f"  • 95% VaR: {var95}\n"
            f"  • Tracking error: {te}\n"
            f"  • Breach probability: {br}\n"
        )


Parameters CSV selected: /Users/teacher/Library/CloudStorage/Dropbox/Learning/Code/Portable Alpha-Extension Model/parameters.csv
Please select the INDEX CSV (monthly total returns).
Index CSV selected: /Users/teacher/Library/CloudStorage/Dropbox/Learning/Code/Portable Alpha-Extension Model/sp500tr_fred_divyield.csv
Loaded 663 months from the raw index CSV.
Using idx_series from 2010-01-01 to 2020-12-01 (n=132)
Using idx_ref from 2010-01-01 to 2014-12-01 (n=60)
Analysis-window: μ_idx = 0.0123, σ_idx = 0.0425
Reference-window: σ_ref = 0.0391
Margin-backing needed for $1 b beta: $468.6 mm
After backing beta ($468.6 mm) and in-house PA ($3000.0 mm), internal cash leftover = $531.4 mm.
Exported results to Outputs.xlsx

=== Summary Table (Human-Friendly) ===

    Config Annual Return (%) Annual Volatility (%) 95%-VaR (%) Tracking Error (%) Breach Probability (%)
      Base              4.4%                  0.8%        3.2%               0.7%                   0.0%
ExternalPA              0.

# Portable Alpha + Active Extension Model Specification

Below is a comprehensive description of the updated portable‐alpha + active‐extension model, ready to paste into a Markdown cell. Every section is clearly labeled, and all equations use LaTeX delimiters.

---

## 1. Purpose and High-Level Overview

**Goal:**  
Construct a Monte Carlo framework that allocates a fixed pool of capital (e.g. \$1 b) across three “sleeves” (Internal, External Portable-Alpha, and Active Extension), simulates joint returns on Index, In-House α, Extension α, and External PA α, and then reports portfolio metrics (annual return, volatility, VaR, tracking error, breach probability).

Key innovations vs. a simpler portable-alpha model:  
1. **Separate “reference period”** used to compute index volatility σₙ, which in turn determines the cash/margin needed to synthetically hold 1:1 index exposure.  
2. **Three explicit buckets** whose dollar-amounts sum to \$ 1 b, avoiding any double-counting of β + α exposures.  
3. **Active Extension bucket** that can be “150/50” or “170/70” long/short, specified by an “Active share (%)” input. By default, we assume 150/50 (i.e. Active share = 50 %) unless the user overrides.

Everything ultimately flows into a set of formulas—one per bucket—that map monthly draws of
\[
(r_{\beta},\,r_{H},\,r_{E},\,r_{M}) 
\quad\text{and}\quad
f_t
\]
into portfolio returns.

---

## 2. Core Assumptions and Variables

1. **Index (β) returns**  
   - We load a historical time series of monthly total returns on the S&P 500 TR (or whichever index) from a CSV.  
   - We partition that series into:  
     1. A **reference window** (e.g. 2010 – 2014) used to compute “reference volatility” σₙ.  
     2. An **analysis window** (e.g. 2015 – 2020) used to compute the actual mean (μₙ) and volatility (σₙ) that drive our Monte Carlo draws.

2. **Three α-streams** (simulated jointly with β)  
   - **In-House α** \($r_H$\):  
     - Mean = μ_H/12  
     - Vol = σ_H / √12  
     - Correlation ρ_{β,H} with β.  
   - **Extension α** \($r_E$\):  
     - Mean = μ_E/12  
     - Vol = σ_E / √12  
     - Correlation ρ_{β,E} with β.  
   - **External PA α** \($r_M$\):  
     - Mean = μ_M/12  
     - Vol = σ_M / √12  
     - Correlation ρ_{β,M} with β.

3. **Financing spread** \($f_t$\)  
   - A month-by-month random draw around a drift (financing_mean/12) with vol (financing_vol/12) and occasional jumps of size (spike_factor × (financing_vol/12)), happening with probability spike_prob.  
   - In each month, any bucket that holds \((r_{\beta} − f_t)\) is charged that financing cost.

4. **Total fund capital** (in millions, default = 1000)  
   - We allocate exactly \$ 1 b across three buckets (plus any residual “cash-leftover” after margin).

5. **Standard-deviation multiple** (sd_of_vol_mult, default = 3)  
   - “To hold \$ 1 b of index exposure, you must keep aside cash = σₙ × (sd_of_vol_mult) × \$ 1 b.”  
   - That cash is the **internal beta-backing** or “margin cash,” needed for futures/swaps.

6. **Three capital buckets** (all in \$ mm, must sum to 1000)  
   1. **External PA capital** \($X$\)  
      - Manager takes \$ X m; buys \$ X m of index (β) and \((external_pa_alpha_frac × X m)\) of α.  
      - Default α fraction = 50 % (\(\theta_{\mathrm{ExtPA}}=0.50\)).  
   2. **Active Extension capital** \($Y$\)  
      - Manager runs a long/short portfolio with **Active share** \(S\).  
      - By default, “150/50” means \(S=0.50\) (i.e. 150 % long, 50 % short → net 100 %).  
   3. **Internal PA capital** \($Z$\)  
      - Runs in-house α; the remainder of internal cash (beyond margin) is used here.

7. **Internal beta backing** \($W$\) (computed, not user-entered)  
   \[
     W = \sigma_{\text{ref}} \times (\mathrm{sd\_of\_vol\_mult}) \times 1000 \quad (\text{\$ mm}).
   \]
   - That cash sits in reserve to back a \$ 1 b index position via futures/swaps.  
   - Because the external PA and active-extension managers each hold index exposure “inside” their \$ X m or \$ Y m, **you do not hold margin for that portion**. You only hold \(W\) for the total \$ 1 b.

---

## 3. Capital-Allocation Equations

1. **Check**:  
   \[
     X + Y + Z \;=\; 1000 \quad(\text{\$ mm}),
   \]  
   where  
   - \(X = \text{external\_pa\_capital},\)  
   - \(Y = \text{active\_ext\_capital},\)  
   - \(Z = \text{internal\_pa\_capital}.\)

2. **Margin (internal beta backing)**:  
   \[
     W = \sigma_{\text{ref}} \times (\mathrm{sd\_of\_vol\_mult}) \times 1000 \quad (\text{\$ mm}).
   \]

3. **Internal cash leftover (runs In-House PA)**:  
   \[
     \text{internal\_cash\_leftover} 
     = 1000 - W - Z \quad (\text{\$ mm}).
   \]

   - If \(W + Z > 1000\), the capital structure is infeasible (you cannot hold margin + in-house PA + external buckets all on \$ 1 b).

---

## 4. Return Equations

We simulate, for each month \(t\):

\[
(r_{\beta,t},\,r_{H,t},\,r_{E,t},\,r_{M,t}) 
\;\sim\;\text{MVN}\bigl([\mu_{\beta},\,\mu_H,\,\mu_E,\,\mu_M],\,\Sigma\bigr),
\]
with
- \(\mu_{\beta} = \mu_{\text{idx}}\) (monthly mean from analysis window),  
- \(\mu_H = \frac{\mu_H^{(\text{annual})}}{12}\),  
- \(\mu_E = \frac{\mu_E^{(\text{annual})}}{12}\),  
- \(\mu_M = \frac{\mu_M^{(\text{annual})}}{12}\).  

Covariance \(\Sigma\) built from:  
- \(\sigma_{\beta} = \sigma_{\text{ref}}\) (monthly vol from reference window),  
- \(\sigma_H = \sigma_H^{(\text{annual})}/\sqrt{12}\),  
- \(\sigma_E = \sigma_E^{(\text{annual})}/\sqrt{12}\),  
- \(\sigma_M = \sigma_M^{(\text{annual})}/\sqrt{12}\),  
- Pairwise correlations \(\rho_{\beta,H},\,\rho_{\beta,E},\,\rho_{\beta,M},\,\rho_{H,E},\,\dots\).  

Additionally, each month we draw a financing cost:
\[
f_t = \frac{\text{financing_mean}}{12} + \varepsilon_t,\quad
\varepsilon_t \sim \mathcal{N}\bigl(0,\;(\tfrac{\text{financing_vol}}{12})^2\bigr),
\]
with probability \(\text{spike_prob}\) of a jump \(=\text{spike_factor} \times \frac{\text{financing_vol}}{12}\).

---

### 4.1. Base (All In-House) Strategy

\[
R_{\text{Base},t}
= \; (r_{\beta,t} - f_t)\,\times\,w_{\beta_H}
\;+\; r_{H,t}\,\times\,w_{\alpha_H}.
\]
By default, \(w_{\beta_H} = 0.50\) and \(w_{\alpha_H} = 0.50\).

---

### 4.2. External PA Strategy

- Capital allocated: \(X = \text{external_pa_capital}\).  
- Manager buys \$ X m of index (β) and allocates \(\theta_{\mathrm{ExtPA}} = \text{external_pa_alpha_frac}\) of that \$ X m to α.  

Return formula:
\[
R_{\text{ExtPA},t}
= \underbrace{\frac{X}{1000}}_{w_{\beta}^{\text{ExtPA}}}\,(r_{\beta,t} - f_t)
\;+\;\underbrace{\tfrac{X}{1000} \,\times\,\theta_{\mathrm{ExtPA}}}_{w_{\alpha}^{\text{ExtPA}}}\;(r_{M,t}).
\]
- If \(\theta_{\mathrm{ExtPA}} = 0.50\), then half of \$ X m is alpha, half is index.

---

### 4.3. Active Extension Strategy

- Capital allocated: \(Y = \text{active_ext_capital}\).  
- Manager runs a long/short portfolio with **Active share** \(S = \frac{\text{active_share_percent}}{100}\).  
  - E.g. 150/50 → \(S = 0.50\).  
  - 170/70 → \(S = 0.70\).

Return formula:
\[
R_{\text{ActExt},t}
= \underbrace{\frac{Y}{1000}}_{w_{\beta}^{\text{ActExt}}}\,(r_{\beta,t} - f_t)
\;+\;\underbrace{\frac{Y}{1000}\,\times\,S}_{w_{\alpha}^{\text{ActExt}}}\;(r_{E,t}).
\]
- The manager’s long/short is embedded in \(r_{E,t}\).  

---

### 4.4. Internal Margin & Internal PA

Because both external PA and active-extension managers hold their own index exposure, on your books you only need to hold margin for a single \$ 1 b of index. That is:
\[
W = \sigma_{\text{ref}} \times (\mathrm{sd\_of\_vol\_mult}) \times 1000 \quad (\text{\$ mm}).
\]
Then you also decide to run \(Z = \text{internal_pa_capital}\) in-house PA:

- **Internal Beta (margin):**  
  \[
  R_{\text{IntBet},t}
  = \Bigl(\tfrac{W}{1000}\Bigr)\,(r_{\beta,t} - f_t).
  \]
- **Internal PA alpha:**  
  \[
  R_{\text{IntPA},t}
  = \Bigl(\tfrac{Z}{1000}\Bigr)\,(r_{H,t}).
  \]
- **Internal cash leftover:**  
  \[
  \text{internal\_cash\_leftover} = 1000 - W - Z \quad (\text{if positive, earns 0}).
  \]

---

## 5. Putting It All Together in Simulation

1. **Read user inputs** (via `load_parameters()`):
   - Dates: `start_date`, `end_date`, `ref_start_date`, `ref_end_date`
   - Vol/risk: `sd_of_vol_mult`
   - Returns: `financing_mean`, `financing_vol`, `μ_H`, `σ_H`, `μ_E`, `σ_E`, `μ_M`, `σ_M`
   - Correlations: `ρ_{β,H}`, `ρ_{β,E}`, `ρ_{β,M}`, `ρ_{H,E}`, `ρ_{H,M}`, `ρ_{E,M}`
   - Capital buckets: `external_pa_capital`, `external_pa_alpha_frac`, `active_ext_capital`, `active_share_percent`, `internal_pa_capital`
   - Total fund capital (mm): default = 1000

2. **Load index CSV** → `idx_full` (monthly total returns).

3. **Filter**  
   - **`idx_series`** = `idx_full[ start_date : end_date ]` → used for μ_β and σ_β.  
   - **`idx_ref`** = `idx_full[ ref_start_date : ref_end_date ]` → used for σ_ref.

4. **Compute**  
   \[
     \mu_{\beta} = \mathrm{mean}(idx\_series), 
     \quad
     \sigma_{\beta} = \mathrm{std}(idx\_series),
     \quad
     \sigma_{\text{ref}} = \mathrm{std}(idx\_ref).
   \]

5. **Margin-backing**  
   \[
     W = \sigma_{\text{ref}} \times \mathrm{sd\_of\_vol\_mult} \times 1000.
   \]
   If \(W + Z > 1000\), error. Else compute
   \[
     \text{internal\_cash\_leftover} = 1000 - W - Z.
   \]

6. **Build covariance matrix** \(\Sigma\) for \((r_{\beta}, r_H, r_E, r_M)\) using  
   \(\sigma_{\beta} = \sigma_{\text{ref}},\; \sigma_H = \frac{\sigma_H^{(\text{annual})}}{\sqrt{12}},\; \sigma_E = \frac{\sigma_E^{(\text{annual})}}{\sqrt{12}},\; \sigma_M = \frac{\sigma_M^{(\text{annual})}}{\sqrt{12}},\)  
   and correlations.

7. **Monte Carlo draws**:  
   For each of \(N_{\text{SIMULATIONS}}\) trials, simulate a \(T=N_{\text{MONTHS}}\)-month path of \(\,(r_{\beta,t},\,r_{H,t},\,r_{E,t},\,r_{M,t})\) and financing \(f_t\).

8. **Compute monthly returns** for each bucket:
   - **Base**:  
     \[
       R_{\text{Base},t} 
       = (r_{\beta,t} - f_t)\,w_{\beta_H} \;+\; r_{H,t}\,w_{\alpha_H}.
     \]
   - **External PA**:  
     \[
       R_{\text{ExtPA},t} 
       = \bigl(\tfrac{X}{1000}\bigr)(r_{\beta,t} - f_t) 
       \;+\; \bigl(\tfrac{X}{1000}\,\theta_{\mathrm{ExtPA}}\bigr)(r_{M,t}).
     \]
   - **Active Extension**:  
     \[
       R_{\text{ActExt},t} 
       = \bigl(\tfrac{Y}{1000}\bigr)(r_{\beta,t} - f_t) 
       \;+\; \bigl(\tfrac{Y}{1000}\,S\bigr)(r_{E,t}).
     \]
   - **Internal Beta**:  
     \[
       R_{\text{IntBet},t} 
       = \bigl(\tfrac{W}{1000}\bigr)(r_{\beta,t} - f_t).
     \]
   - **Internal PA α**:  
     \[
       R_{\text{IntPA},t} 
       = \bigl(\tfrac{Z}{1000}\bigr)(r_{H,t}).
     \]

   Note: We only report three portfolios—“Base,” “ExternalPA,” and “ActiveExt.” Each one compounds its own monthly returns for a 12-month horizon:
   \[
     R_{\text{bucket}}^{\text{(year)}} 
     = \prod_{t=1}^{12} (1 + R_{\text{bucket},t}) - 1.
   \]

9. **Compute performance metrics** for each portfolio’s annual returns:
   - **Ann Return** = sample mean.  
   - **Ann Vol** = sample standard deviation.  
   - **VaR 95%** = 5th percentile.  
   - **Tracking Error** = std of (bucket_return − index_return).  
   - **Breach Probability** = % of months (in the first sim path) where \((r_{\text{bucket},t} < -\,\mathrm{buffer\_multiple}\times\sigma_{\beta})\).

10. **Export**  
    - **Inputs sheet:** all parameters (dates, vol caps, bucket sizes, α fractions, active share, σ_ref, W, internal cash leftover, etc.).  
    - **Summary sheet:** metrics for “Base,” “ExternalPA,” and “ActiveExt.”  
    - **Raw returns sheets:** monthly paths for each bucket (first simulation) so users can inspect breach months.

---

## 6. Input Parameters Summary

Below is a consolidated list of every input variable that must appear in the “friendly” CSV:

1. **Date ranges**  
   - `Start date` → `start_date` (analysis window begin).  
   - `End date` → `end_date` (analysis window end).  
   - `Reference start date` → `ref_start_date` (for σ_ref).  
   - `Reference end date` → `ref_end_date` (for σ_ref).  

2. **Financing parameters**  
   - `Annual financing mean (%)` → `financing_mean_annual` (default = 0.50 %).  
   - `Annual financing vol (%)` → `financing_vol_annual` (default = 0.10 %).  
   - `Monthly spike probability` → `spike_prob` (default = 2 %).  
   - `Spike size (σ × multiplier)` → `spike_factor` (default = 2.25).  

3. **In-House PA parameters**  
   - `In-House annual return (%)` → `mu_H` (default = 4.00 %).  
   - `In-House annual vol (%)` → `sigma_H` (default = 1.00 %).  
   - `In-House β` → `w_beta_H` (default = 0.50).  
   - `In-House α` → `w_alpha_H` (default = 0.50).  

4. **Extension α parameters**  
   - `Alpha-Extension annual return (%)` → `mu_E` (default = 5.00 %).  
   - `Alpha-Extension annual vol (%)` → `sigma_E` (default = 2.00 %).  
   - `Active Extension capital (mm)` → `active_ext_capital` (default = 0).  
   - `Active share (%)` → `active_share_percent` (default = 50 % ⇒ a 150/50 program).  

5. **External PA α parameters**  
   - `External annual return (%)` → `mu_M` (default = 3.00 %).  
   - `External annual vol (%)` → `sigma_M` (default = 2.00 %).  
   - `External PA capital (mm)` → `external_pa_capital` (default = 0).  
   - `External PA α fraction (%)` → `external_pa_alpha_frac` (default = 50 %).  

6. **Correlations**  
   - `Corr index–In-House` → `rho_idx_H` (default = 0.05).  
   - `Corr index–Alpha-Extension` → `rho_idx_E` (default = 0.00).  
   - `Corr index–External` → `rho_idx_M` (default = 0.00).  
   - `Corr In-House–Alpha-Extension` → `rho_H_E` (default = 0.10).  
   - `Corr In-House–External` → `rho_H_M` (default = 0.10).  
   - `Corr Alpha-Extension–External` → `rho_E_M` (default = 0.00).  

7. **Capital & risk backing**  
   - `Total fund capital (mm)` → `total_fund_capital` (default = 1000).  
   - `Standard deviation multiple` → `sd_of_vol_mult` (default = 3).  
   - `Internal PA capital (mm)` → `internal_pa_capital` (default = 0).  
   - `Buffer multiple` → `buffer_multiple` (default = 3).  

8. **Legacy/Optional**  
   - `X grid (mm)` → `X_grid_list` (list of X values).  
   - `External manager α fractions` → `EM_thetas_list`.

---

## 7. Output Considerations

1. **Inputs sheet (Excel):**  
   List every single parameter, including:  
   - Date windows (analysis and reference),  
   - Financing parameters,  
   - α-stream parameters,  
   - Correlations,  
   - Capital buckets (X, Y, Z),  
   - SD multiple, margin backing \(W\), internal cash leftover,  
   - Active share, etc.

2. **Summary sheet (Excel):**  
   For each portfolio (“Base,” “ExternalPA,” “ActiveExt”), show:  
   - Annual Return (%),  
   - Annual Volatility (%),  
   - 95 % VaR (%),  
   - Tracking Error (%),  
   - Breach Probability (%).

3. **Raw returns sheets (Excel):**  
   Monthly paths for each bucket (first simulation), so users can inspect “breach” months where \(R_{t} < -(\text{buffer_multiple} × σ_{\beta})\).

4. **Console output:**  
   A “human‐friendly” summary, e.g.:  
   > For “ExternalPA (X = 300, 50 % α)”:  
   > • Expected annual return: 10.2 %  
   > • Annual volatility: 12.3 %  
   > • 95 % VaR: −3.4 %  
   > • Tracking error: 8.7 %  
   > • Breach probability: 2.0 %.

---

## 8. Intuition Behind Key Pieces

1. **Why a separate reference period?**  
   - If you measure index volatility over the same window you analyze (e.g. 2015–2020), you capture “current regime” vol. Often, managers prefer a longer/different window (e.g. 2010–2014) to gauge typical funding volatility. That reference σₙ, times a multiple (e.g. 3×), tells you how much cash to set aside to back \$ 1 b of index exposure.

2. **Why Active share as a percentage?**  
   - A “150/50” program has 150 % long and 50 % short = net 100 %. Its “active share” is reported as 50 %.  
   - If you want “170/70,” then active share = 70 %.  
   - The code converts “Active share (%)” to decimal \(S\). For a 150/50 program, the default is 50 % (\(S = 0.50\)).

3. **Why each bucket’s formula ensures no double-counting**  
   - Whenever you give \$ X m to External PA, that manager holds the index exposure on your behalf. You do not hold margin for that portion. Similarly, the Active Extension manager holds their own index.  
   - On your books, you only need to hold margin for a single \$ 1 b index. That is \(W\).  
   - Once you hand \$ X m to external PA and \$ Y m to active ext, **both managers** hold \((X + Y)\) of index on your behalf. So your margin \(W\) backs the *entire* \$ 1 b, not just the “leftover” portion.

---

## 9. Step-by-Step Implementation Checklist

1. **Read and parse user parameters** (dates, vols, α fractions, active share, capital buckets, etc.).  
2. **Load index CSV** → `idx_full`.  
3. **Filter** → `idx_ref` for σ_ref; `idx_series` for μ_β and σ_β.  
4. **Compute**:  
   \[
     μ_β = \mathrm{mean}(idx\_series), 
     \quad
     σ_β = \mathrm{std}(idx\_series), 
     \quad
     σ_{\text{ref}} = \mathrm{std}(idx\_ref).
   \]
5. **Margin-backing**:  
   \[
     W = σ_{\text{ref}} × (\mathrm{sd\_of\_vol\_mult}) × 1000.
   \]
   Check \(W + Z ≤ 1000\). Compute leftover internal cash = \(1000 - W - Z\).

6. **Build covariance matrix** using \((σ_{\text{ref}},\,σ_H/√{12},\,σ_E/√{12},\,σ_M/√{12})\) plus correlations.

7. **Monte Carlo draws**:  
   For each of \(N_{\mathrm{SIM}}\) trials, simulate a path of length \(T = N_{\mathrm{MONTHS}}\) for \((r_{\beta,t},\,r_{H,t},\,r_{E,t},\,r_{M,t})\) and financing \(f_t\).

8. **Compute monthly returns**:
   - **Base**:  
     \[
       R_{\text{Base},t} = (r_{\beta,t} - f_t)\,w_{\beta_H} + r_{H,t}\,w_{\alpha_H}.
     \]
   - **External PA**:  
     \[
       R_{\text{ExtPA},t}
       = \Bigl(\tfrac{X}{1000}\Bigr)(r_{\beta,t} - f_t)
       \;+\;\Bigl(\tfrac{X}{1000}\,\theta_{\mathrm{ExtPA}}\Bigr)(r_{M,t}).
     \]
   - **Active Extension**:  
     \[
       R_{\text{ActExt},t}
       = \Bigl(\tfrac{Y}{1000}\Bigr)(r_{\beta,t} - f_t)
       \;+\;\Bigl(\tfrac{Y}{1000}\,S\Bigr)(r_{E,t}).
     \]
   - **Internal Beta**:  
     \[
       R_{\text{IntBet},t} 
       = \Bigl(\tfrac{W}{1000}\Bigr)(r_{\beta,t} - f_t).
     \]
   - **Internal PA α**:  
     \[
       R_{\text{IntPA},t} 
       = \Bigl(\tfrac{Z}{1000}\Bigr)(r_{H,t}).
     \]

9. **Aggregate monthly → annual returns** for “Base,” “ExternalPA,” “ActiveExt.”  
10. **Compute metrics**:  
    - Ann Return, Ann Vol, VaR 95, Tracking Error, Breach Probability.  
11. **Export** Inputs, Summary, Raw returns to Excel + print narrative.

---